# Q-Learning
- 탐험(Exploration)을 하면서도 최적의 정책을 학습이 가능함(상대적으로 SARSA에 비해서)
- Off-Policy
    - SARSA와 같은 on-policy는 이전 정책으로부터 얻은 샘플을 재사용할 수 없고, 자신의 샘플로 자신을 업데이트하므로 문제점이 존재할 수 있음
    - 정책이 복수개이므로 behavior policy(현재의 예시에선 입실론 그리디)로 샘플을 수집하고 target policy로 최적의 정책을 학습
- 학습하는 정책(TD) : 탐욕정책 -> exploitation

$$q(s, a) = q(s, a) + \alpha \big( r + \gamma\ max_{a'} q(s', a') - q(s, a)  \big )$$

- 행동하는 정책 : $\epsilon$-greedy -> exploration
    - 종류 : $\epsilon$-greedy, BoltMann, Bayesian, ...
    - 아래의 예는 SARSA와 같은 입실론 그리디
    
$$\pi(s) = \begin{cases} 
    a^* = argmax_{a \in A} q(s, a), 1 - \epsilon \\
    \text{random action}, \ \epsilon 
\end{cases}$$

## Q-Learning의 학습 순서
1. 상태 s에서 행동하는 a는 행동 정책(입실론탐욕)으로 선택
2. 환경으로부터 다음 상태 s'와 보상 r을 받음
3. 벨만 최적 방정식을 통해 q(s, a)를 업데이트
    - 학습하는 정책 : 탐욕정책

$$q(s, a) = q(s, a) + \alpha \big( r + \gamma\ max_{a'} q(s', a') - q(s, a)  \big )$$

- 벨만 최적 방정식을 이용하므로 s'에서 a'를 선택하지 않음

In [1]:
import numpy as np
import random
from environment import Env
from collections import defaultdict
import time

In [2]:
class QLearningAgent:
    def __init__(self, actions):
        # 행동 = [0, 1, 2, 3] 순서대로 상, 하, 좌, 우
        self.actions = actions
        self.learnging_rate = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])
        
    # <s, a, r, s'> 샘플로부터 큐함수 업데이트(exploitation), greedy
    def learn(self, state, action, reward, next_state):
        q_1 = self.q_table[state][action]
        # 벨만 최적 방정식 
        q_2 = reward + self.discount_factor * max(self.q_table[next_state])
        self.q_table[state][action] += self.learnging_rate * (q_2 - q_1)
        
    # 큐함수로 입실론 탐욕 정책에 따라서 행동을 반환한다
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 무작위 행동 반환
            action = np.random.choice(self.actions)
        else:
            # 큐함수에 따른 행동 반환
            state_action = self.q_table[state]
            action = self.arg_max(state_action)
        return action
    
    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

### Q-Learning Agent에 따른 정책 학습

In [3]:
EPISODES = 1000

if __name__ == "__main__":
    env = Env()
    agent = QLearningAgent(actions=list(range(env.n_actions)))
    
    for episode in range(EPISODES):
        state = env.reset()
        
        while True:
            env.render()
            
            # 현재 상태에 대한 행동을 선택
            action = agent.get_action(str(state))
            
            # 행동을 취한 후 다음 상태, 보상, 에피소드의 종료 여부를 받아온다
            next_state, reward, done = env.step(action)
            
            # <s, a, r, s'>로 큐함수를 업데이트
            agent.learn(str(state), action, reward, str(next_state))
            state = next_state
            
            # 모든 큐함수를 화면에 표시
            env.print_value_all(agent.q_table)
            
            if done:
                if episode == 30:
                    for y in range(5):
                        for x in range(5):
                            print("x : ", x, ' y :', y, agent.q_table[str([x, y])])
                        print("")
                    time.sleep(100)
                break

x :  0  y : 0 [0.0, 1.9474449511909893e-06, 0.0, 0.0]
x :  1  y : 0 [0.0, 0.0, 1.2461694768832958e-08, 0.0]
x :  2  y : 0 [0.0, -1.0, 0.0, 0.0]
x :  3  y : 0 [0.0, 0.0, 0.0, 0.0]
x :  4  y : 0 [0.0, 0.0, 0.0, 0.0]

x :  0  y : 1 [0.0, 8.392766731440504e-05, 0.0, 1.5836941800000003e-12]
x :  1  y : 1 [0.0, -1.0, 1.0464132339e-09, -2.9701]
x :  2  y : 1 [0.0, 0.0, 0.0, 0.0]
x :  3  y : 1 [0.0, 0.009000000000000001, -1.0, 0.0]
x :  4  y : 1 [0.0, 0.0, 0.0, 0.0]

x :  0  y : 2 [0.0, 0.0028435541426892566, 1.9979487303020747e-06, -1.99]
x :  1  y : 2 [0.0, 0.0, 0.0, 0.0]
x :  2  y : 2 [0.0, 0.0, 0.0, 0.0]
x :  3  y : 2 [0.0, 0.0, 1.99, 0.0]
x :  4  y : 2 [0.0, 0.0, 0.0, 0.0]

x :  0  y : 3 [0.0, 0.0, 0.00048380020214232284, 0.0782519567627358]
x :  1  y : 3 [-1.0, 7.290000000000002e-07, 0.0, 1.6363086690409852]
x :  2  y : 3 [18.20930624027691, 0.0, 0.009970278279254853, 0.0]
x :  3  y : 3 [0.0, 0.0, 0.12594748082284032, 0.0]
x :  4  y : 3 [0.0, 0.0, 0.0, 0.0]

x :  0  y : 4 [0.0, 0.0, 0.0,